# AustinTrafficPy

Eric Staveley  MWSa

## Analysis

* xxx

* yyy

* zzz

In [30]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import datetime

# Import API (api_key) key from api_keys.py
#from api_keys import api_key
#from api_keys import gkey

# Incorporated citipy to determine city based on latitude and longitude
#from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/aus_traffic.csv"

# Range of latitudes and longitudes
#lat_range = (-90, 90)
#lng_range = (-180, 180)

#get current time
now = datetime.datetime.now()


In [31]:
#we r using the csv though for now...    
#BUT GOT THE QUERY TO WORK TO BRING DOWN THE 100K records!!!

#real deal for looping here:
# Build partial query URL..city to be added later
#query_url = f"{url}appid={api_key}&units={units}&q="

#query_url = "https://data.austintexas.gov/resource/r3af-2r8x.json?$limit=100000"
#print(query_url)

In [32]:
#response = requests.get(query_url).json()
#print(response)

In [33]:
#print a pretty better json structure display:
#print(json.dumps(response, indent=4, sort_keys=True))


In [34]:
#len(response)    # i could only find a way to get 1000...so using .csv method

In [35]:
#df = pd.DataFrame(response)
#df.head()

In [36]:
##Austin:  30.2672° N, 97.7431° W    per wiki...center
#type(df["published_date"])

In [37]:
#print(df.iloc[0,5])
#type(df.iloc[0,5])

In [38]:
#dateconv_df = pd.to_datetime(df['published_date'])

In [39]:
#dateconv_df.head()

In [40]:
#df['published_date'] =  pd.to_datetime(df['published_date'])
#df.head()
#df['traffic_report_status_date_time'] =  pd.to_datetime(df['traffic_report_status_date_time'])
#df.head()
#len(df)

# OUR ACTUAL ETL  STARTS HERE

In [41]:
#csv method...since we could get a larger dataset
data_file = "Resources/Real-Time_Traffic_Incident_Reports.csv"
#data_file = "//usaussfs2008.ad.global/home$/rstaveley/new_dir/Real-Time_Traffic_Incident_Reports.csv"
df_from_csv = pd.read_csv(data_file)
df_from_csv.head(20)

print(f"Raw data record count: {len(df_from_csv)}")


Raw data record count: 98324


In [42]:
#clean up the data

#get rid of the NaNs found in Location
#19	9BD7909EA32C89F68E82ED0CBE1F30A2E1D72B1C_15323...	07/23/2018 09:39:17 PM +0000	zSTALLED VEHICLE	NaN	0.000000	0.000000	3339 W BRAKER LN	ARCHIVED	07/23/2018 10:15:02 PM +0000

print(f"Non-NA values per column:\n{df_from_csv.count()}")   #show the non-NA cells for each field

#remove the rows with any empty cells
df_from_csv_clean = df_from_csv.dropna(how='any')

print(f"Total of rows with complete data now: {len(df_from_csv_clean)}")

print("Removing invalid longitude and latitude...")
#ensure lat and long are valid.  -90<=lat<=90     -180<=lon<=180
#lat
row_condition = df_from_csv_clean["Latitude"] <= 90
filtered_df = df_from_csv_clean.loc[row_condition]
row_condition = filtered_df["Latitude"] >= -90
filtered_df = filtered_df.loc[row_condition]

#long
row_condition = filtered_df["Longitude"] <= 180
filtered_df = filtered_df.loc[row_condition]
row_condition = filtered_df["Longitude"] >= -180
filtered_df = filtered_df.loc[row_condition]

print(f"Total number of rows with valid lat/long data: {len(filtered_df)}")
df_from_csv_clean = filtered_df   #re-assign filtered df back to our orig name

df_from_csv_clean.dtypes

Non-NA values per column:
Traffic Report ID    98324
Published Date       98324
Issue Reported       98324
Location             98045
Latitude             98245
Longitude            98245
Address              98324
Status               96631
Status Date          98324
dtype: int64
Total of rows with complete data now: 96358
Removing invalid longitude and latitude...
Total number of rows with valid lat/long data: 96352


Traffic Report ID     object
Published Date        object
Issue Reported        object
Location              object
Latitude             float64
Longitude            float64
Address               object
Status                object
Status Date           object
dtype: object

In [43]:
#df_from_csv_clean.count() 

In [44]:
#convert the date strings in the DF to valid datetime types
df_from_csv_clean['Published Date'] =  pd.to_datetime(df_from_csv_clean['Published Date'])
#df_from_csv_clean.head()
df_from_csv_clean['Status Date'] =  pd.to_datetime(df_from_csv_clean['Status Date'])
df_from_csv_clean.head()

,Traffic Report ID,Published Date,Issue Reported,Location,Latitude,Longitude,Address,Status,Status Date
0,C163BCD1CF90C984E9EDA4DBA311BCA369A7D1A1_15288...,2018-06-13 06:35:59,Crash Service,"(30.283797,-97.741906)",30.283797,-97.741906,W 21ST ST & GUADALUPE ST,ARCHIVED,2018-06-13 09:00:03
1,2664C206999E7419517EE39E177797576A52201B_15381...,2018-09-29 00:52:58,Traffic Hazard,"(30.380525,-97.737873)",30.380525,-97.737873,0 Mopac Sb To Research Sb Ramp,ARCHIVED,2018-09-29 01:35:03
2,5BB246A3223A89C7FB9B2EED9A2EBDA82141406D_15438...,2018-12-03 17:49:02,Crash Service,"(30.278372,-97.671971)",30.278372,-97.671971,5800 Techni Center Dr,ARCHIVED,2018-12-03 18:30:03
3,6B20382196FB454E9FD06A33E60142902A2F0706_15288...,2018-06-13 10:15:36,Traffic Hazard,"(30.339593,-97.700963)",30.339593,-97.700963,400-717 E ANDERSON LN EB,ARCHIVED,2018-06-13 11:20:03
4,7FD2528A7609AAECB6AF825AC275F98CFDFFF8AD_15382...,2018-09-29 12:47:30,Traffic Hazard,"(30.281659,-97.728551)",30.281659,-97.728551,0 N Ih 35 Ud Sb To Mlk Ramp,ARCHIVED,2018-09-29 13:00:03


In [45]:
#peek at the earliest and latest date in our DF
earliest_date = min(df_from_csv_clean["Published Date"])
print(f"Earliest date: {earliest_date}")
latest_date = max(df_from_csv_clean["Published Date"])
print(f"Latest date: {latest_date}")
type(latest_date)     #confirm the type

df_from_csv_clean.dtypes


Earliest date: 2017-09-26 16:11:00
Latest date: 2019-03-06 02:37:18


Traffic Report ID            object
Published Date       datetime64[ns]
Issue Reported               object
Location                     object
Latitude                    float64
Longitude                   float64
Address                      object
Status                       object
Status Date          datetime64[ns]
dtype: object

In [46]:
#use only 2018 data now
#keep only 2018 data
row_condition = df_from_csv_clean["Published Date"] <= "2018-12-31 23:59:59"
filtered_df = filtered_df.loc[row_condition]
row_condition = filtered_df["Published Date"] >= "2018-01-01 00:00:01"
filtered_df = filtered_df.loc[row_condition]

print(f"Total number of rows with valid lat/long data: {len(filtered_df)}")
df_from_csv_clean = filtered_df   #re-assign filtered df back to our orig name


Total number of rows with valid lat/long data: 68244


In [47]:
#look at types of accidents and counts
df_from_csv_clean["Issue Reported"].value_counts()

Traffic Hazard                20272
Crash Urgent                  15411
Crash Service                 11343
COLLISION                      4793
Traffic Impediment             4745
TRFC HAZD/ DEBRIS              4060
zSTALLED VEHICLE               3008
COLLISION WITH INJURY          1573
LOOSE LIVESTOCK                1471
COLLISN/ LVNG SCN              1000
COLLISION/PRIVATE PROPERTY      250
VEHICLE FIRE                    145
BLOCKED DRIV/ HWY                93
ICY ROADWAY                      21
BOAT ACCIDENT                    20
AUTO/ PED                        15
TRAFFIC FATALITY                 11
FLEET ACC/ INJURY                 9
N / HZRD TRFC VIOL                2
HIGH WATER                        1
COLLISN / FTSRA                   1
Name: Issue Reported, dtype: int64

In [49]:
import gmaps

# Google developer API key
from api_keys import gkey

# Access maps with unique API key
gmaps.configure(api_key=gkey)


In [50]:
# code to take the lat and long and make a list for gmap

aus_lat = df_from_csv_clean["Latitude"]    #series
aus_long = df_from_csv_clean["Longitude"]    #series

aus_lat_subset = aus_lat.head(1000)   #make a subset to test
aus_long_subset = aus_long.head(1000)  # make a subset to test

#print(type(aus_lat_subset))     #its a series
#print(aus_lat_subset[0])        #whats the value?
#print(type(aus_lat_subset[0]))    #its a string

#print(aus_lat_subset)   #show the whole lat subset

#using zip function to zip the lat and long together to a list with each element being a tuple
aus_lat_long_list_subset = list(zip(aus_lat_subset, aus_long_subset))
aus_lat_long_list = list(zip(aus_lat, aus_long))

#print(aus_lat_long_list_subset)
#type(aus_lat_long_list_subset[0])    #each element is a tuple...yay!

In [51]:
# Customize the size of the figure      was   400 x 300
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [52]:
# Assign the marker layer to a variable  ..      DOING THE SUBSET HERE... TAKES ABOUT 10sec  

#help(gmaps.marker_layer)

markers = gmaps.marker_layer(aus_lat_long_list_subset)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [53]:
#extracting the marker image is a bit different than plt.savefig....basically
#gmaps only lets you save the .html  OR  you can manually dload the fig 
#within the display frame
#per https://jupyter-gmaps.readthedocs.io/en/latest/export.html
#
#to export:
#from ipywidgets.embed import embed_minimal_html
#embed_minimal_html('exported_filename_here.html', views=[fig])
#
#to open the file:
#python -m http.server 8080
#Navigate to http://0.0.0.0:8080/exported_filename.html and you should see the export!


In [54]:
#help(gmaps.marker_layer)

In [55]:
fig = gmaps.figure(map_type='HYBRID')
heatmap_layer = gmaps.heatmap_layer(aus_lat_long_list_subset)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))